# TSPTW

In the traveling salesperson problem with time windows (TSPTW), we are given a set of locations $N = \{ 0, ..., n-1 \}$.
The salesperson starts from the depot $0$, visit each customer $i \in \{ 1, ..., n-1 \}$ exactly once, and returns to the depot.
The traveling time from $i$ to $j$ is $c_{ij}$.
Each customer $i$ must be visited within time window $[a_i, b_i]$, and the salesperson must wait until $a_i$ if arriving at $i$ before $a_i$.
The objective is to minimize the total travel time (not including the waiting time).

## DP Formulation

Let $U \subseteq N$ be the set of unvisited customers, $i \in N$ be the current location of the salesperson, and $t$ be the current time.
Let $V(U, i, t)$ be the optimal cost to visit customers $U$ and return to the depot starting from $i$ with time $t$.
When customer $j \in U$ is visited next, the problem is reduced to visiting customers $U \setminus \{ j \}$ from location $j$ at time $\max \{ t + c_{ij}, a_j \}$.
When all customers are visited, the salesperson must return to the depot from location $i$. We have the following DP formulation.

$$
\begin{align}
    \text{compute } & V(N \setminus \{ 0 \}, 0, 0) \\ 
    & V(U, i, t) = \begin{cases}
         \min\limits_{j \in U : t + c_{ij} \leq b_j} c_{ij} + V(U \setminus \{ j \}, j, \max \{ t + c_{ij}, a_j \})  & \text{else if } U \neq \emptyset \\
         c_{i0} + V(U, 0, t + c_{i0}) & \text{else if } U = \emptyset \land i \neq 0 \\
         0 & \text{else if } U = \emptyset \land i = 0.
    \end{cases}.
\end{align}
$$

The earliest arrival time at customer $j$ is $t + c_{ij}$ (assuming the triangle inequality). If $\exists j \in U, t + c_{ij} > b_j$, the state does not lead to a solution.

$$
    V(U, i, t) = \infty \text{ if } \exists j \in U, t + c_{ij} > b_j.
$$

When two states $(U, i, t)$ and $(U, i, t')$ have the same set of unvisited customers $U$ and the same location $i$ and $t \leq t'$, $(U, i, t)$ leads to a better solution. Therefore,

$$
    V(U, i, t) \leq V(U, i, t') \text{ if } t \leq t'.
$$

The lowest possible travel time to visit customer $j$ is $\min_{k \in N \setminus \{ j \}} c_{kj}$.
Because we need to visit all customers in $U$, the total travel time is at least $\sum_{j \in U} \min_{k \in N \setminus \{ j \}} c_{kj}$. Furthermore, if the current location $i$ is not the depot, we need to visit the depot.
Therefore,

$$
    V(U, i, t) \geq \sum_{j \in (U \cup \{ 0 \}) \setminus \{ i \} } \min_{k \in N \setminus \{ j \}} c_{kj}.
$$

Similarly, we need to depart from each customer in $U$ and the current location $i$ if $i$ is not the depot.
Therefore,

$$
    V(U, i, t) \geq \sum_{j \in (U \cup \{ i \}) \setminus \{ 0 \} } \min_{k \in N \setminus \{ j \}} c_{jk}.
$$

## Install DIDPPy

In [1]:
!pip install didppy

## Data

In [2]:
# Number of locations
n = 4
# Ready time
a = [0, 5, 0, 8]
# Deadline
b = [100, 16, 10, 14]
# Travel time
c = [
    [0, 3, 4, 5],
    [3, 0, 5, 4],
    [4, 5, 0, 3],
    [5, 4, 3, 0],
]

## Modeling

In [3]:
import didppy as dp

model = dp.Model()

customer = model.add_object_type(number=n)

# U
unvisited = model.add_set_var(object_type=customer, target=list(range(1, n)))
# i
location = model.add_element_var(object_type=customer, target=0)
# t
time = model.add_int_resource_var(target=0, less_is_better=True)

travel_time = model.add_int_table(c)

for j in range(1, n):
    visit = dp.Transition(
        name="visit {}".format(j),
        cost=travel_time[location, j] + dp.IntExpr.state_cost(),
        preconditions=[unvisited.contains(j)],
        effects=[
            (unvisited, unvisited.remove(j)),
            (location, j),
            (time, dp.max(time + travel_time[location, j], a[j])),
        ],
    )
    model.add_transition(visit)

return_to_depot = dp.Transition(
    name="return",
    cost=travel_time[location, 0] + dp.IntExpr.state_cost(),
    effects=[
        (location, 0),
        (time, time + travel_time[location, 0]),
    ],
    preconditions=[unvisited.is_empty(), location != 0],
)
model.add_transition(return_to_depot)

model.add_base_case([unvisited.is_empty(), location == 0])

for j in range(1, n):
    model.add_state_constr(
        ~unvisited.contains(j) | (time + travel_time[location, j] <= b[j])
    )

min_to = model.add_int_table(
    [min(c[k][j] for k in range(n) if k != j) for j in range(n)]
)

model.add_dual_bound(min_to[unvisited] + (location != 0).if_then_else(min_to[0], 0))

min_from = model.add_int_table(
    [min(c[j][k] for k in range(n) if k != j) for j in range(n)]
)

model.add_dual_bound(
    min_from[unvisited] + (location != 0).if_then_else(min_from[location], 0)
)

## Solving

In [4]:
solver = dp.CABS(model, quiet=True)
solution = solver.search()

print("Transitions to apply:")
print("")

for t in solution.transitions:
    print(t.name)

print("")
print("Cost: {}".format(solution.cost))

Transitions to apply:

visit 2
visit 3
visit 1
return

Cost: 14
